<a href="https://colab.research.google.com/github/madhurendra0089/Alzheimer-s-Disease-Classification/blob/main/Preprocessing_MRI_%26_fMRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3D MRI Image

In [ ]:
def process_scan(path):
    """Read and extract volume"""
    img = nib.load(path)              # Get raw data
    img = img.get_fdata()             #Extract Image Data

    """Normalize the volume"""
    img = (img-np.min(img))/(np.max(img)-np.min(img))

    return img


def extract_features(image):
    image = process_scan(image)
    x=np.unique(image)
    image = (image-x[0]/x[-1]-x[0])*255
    # Calculate the intensity histogram of the image
    histogram = np.histogram(image, bins=256)

    # Return the intensity histogram as a feature vector
    return histogram[0]

def tunnel(_dir, n):

    """preprocess one folder"""
    _list = [_dir+x+temporal_image for x in os.listdir(_dir)]
    _image = np.array([extract_features(i) for i in _list])
    _labels = np.array([n for _ in range(len(_list))])
    return _image, _labels

def bigger_tunnel():
    """preprocess parent folder"""

    #Process each folder through tunnel function
    ad_image, ad_labels   = tunnel(dir_AD, 1)
    nc_image, nc_labels   = tunnel(dir_NC, 0)

    #Concatenate to make complete set of train, test or validation
    x = np.concatenate(( ad_image[:],  nc_image[:]),axis=0)
    y = np.concatenate((ad_labels[:], nc_labels[:]), axis=0)

    return x,y
    #######################

def tunnel():
    """preprocess parent folder"""

    #Process each folder through tunnel function
    ad_images   = [nib.load(dir_AD+x+temporal_image).get_fdata() for x in os.listdir(dir_AD)]
    nc_images   = [nib.load(dir_NC+x+temporal_image).get_fdata() for x in os.listdir(dir_NC)]
    mci_images  = [nib.load(dir_MCI+x+temporal_image).get_fdata() for x in os.listdir(dir_MCI)]
    #Concatenate to make complete set of train, test or validation
    x = np.concatenate(( ad_images[:],  nc_images[:], mci_images[:] ),axis=0)
    #y = np.concatenate((ad_labels[:], nc_labels[:], mci_labels), axis=0)

    return x

x =tunnel()

## Connectivity Measure Generation for fMRI Image


In [ ]:
from nilearn import datasets, plotting, image, maskers
from nilearn.connectome import ConnectivityMeasure
import numpy as np

# Step 1: Fetch atlas and plot
msdl_atlas = datasets.fetch_atlas_msdl(data_dir=path)
plotting.plot_prob_atlas(msdl_atlas.maps)

# Step 2: Load the temporal image and extract time series
temporal_image_path = path + 'AD/Sub_003/temporal_image.nii.gz'
temporal_image = image.load_img(temporal_image_path)

masker = maskers.NiftiMapsMasker(msdl_atlas.maps, resampling_target='data', detrend=True).fit()
roi_time_series = masker.transform(temporal_image)

# Step 3: Compute the connectivity matrix
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([roi_time_series])[0]

# Step 4: Zero out the diagonal
np.fill_diagonal(correlation_matrix, 0)

# Step 5: Visualize the correlation matrix
plotting.plot_matrix(correlation_matrix, labels=msdl_atlas.labels, vmax=0.8, vmin=-0.8, colorbar=True)

# Step 6: Flatten the lower triangular portion of the connectivity matrix into a 1D array
lower_triangular_indices = np.tril_indices_from(correlation_matrix, k=-1)
flattened_connectivity = correlation_matrix[lower_triangular_indices]

# Step 7: Output the flattened and scaled data
print(flattened_connectivity)


## Intensity Histogram for MRI Image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Flatten the 3D MRI image into a 1D array
flattened_image = mri_image.flatten()

# Calculate the histogram for intensities from 0 to 255
histogram, bin_edges = np.histogram(flattened_image, bins=np.arange(257), range=(0, 255))

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.bar(bin_edges[:-1], histogram, width=1, color='gray', alpha=0.7)
plt.title("Intensity Histogram of 3D MRI Image")
plt.xlabel("Intensity Value")
plt.ylabel("Number of Pixels")
plt.grid(True)
plt.show()
